In [1]:
import subprocess as sp
import os
import shutil
import time
import multiprocessing as mp
from glob import glob
import pandas as pd

In [2]:
pbs = '''
#!/bin/bash
########################################################
##                                                     #
## A Simple PBS command file                           #
## Edit&submit this file using   \"qsub pbs_serial.cmd\" #
## Check your job with           \"qstat\"               #
## Kill your job with            \"qrm\" or \"qdel\"       #
##                                                     #
########################################################
#
# Specify time limit of your job. (HH:MM:SS)
PBS -l walltime=00:20:00
PBS -l nodes=1:procs=1

# Give your job a name so it can be found readily with qstat.
PBS -N {}

# mail alert at the (e)nd of execution
#PBS -m e
#PBS -M your_email_address@ae.metu.edu.tr

# Standard error output merge to the standard output file.
# PBS -j oe
# Specify the file name for the standard output/error
# PBS -o file

# Enter your work directory under /cfd
cd /cfd/{}
cp ../../xfoil .

# execute your serial program
./{}
'''

In [3]:
re_min = 1000000
re_max = 8000005
num_core = 4

def xfoil(params_list):

    cur_dir = params_list[0]
    move_dir = params_list[1]
    Re = params_list[2]
    af_name = params_list[3]
    alpha_min = params_list[4]
    alpha_max = params_list[5]
    alpha_increment = params_list[6]

    with open(af_name[:-3]+'.run', 'w') as fid2:
        fid2.write('load ' + af_name + '\n')
        #fid2.write('save {}.af\nY\n'.format(af_name))
        fid2.write('pane\n\n')
        fid2.write('plop\n')
        fid2.write('g\n\noper\niter 1000\n')
        fid2.write('visc {}\n'.format(str(Re)))
        fid2.write('pacc\n\n\n')
        fid2.write('aseq {} {} {}\n'.format(alpha_min, alpha_max, alpha_increment))
        fid2.write('pacc\npwrt\n')
        fid2.write(af_name[:-3] + '_Re_' + str(Re) + '.polar\n\n\nquit\n')

    # commands to be executed
    fid = open(cur_dir + 'missed.txt', 'a')
    t = time.time()
    try:

        xx = sp.Popen(cur_dir + 'xfoil < {}'.format(af_name[:-3]+'.run'), shell=True, stdout=False)
        xx.wait()
    except:
        fid.write(af_name + '\n')
    time.sleep(20)
    th = time.time() - t
    fid.write(str(th) + '\n')

    shutil.move(cur_dir + '{}'.format(af_name[:-3]+'.run'), move_dir)
        #os.remove(cur_dir + '{}'.format(af_name))
    shutil.move(cur_dir + '{}'.format(af_name[:-3] + '_Re_' + str(Re) + '.polar'), move_dir)
        #shutil.copy(cur_dir + '{}'.format(af_name), move_dir)


    fid.close()
    return

def create_run_files(params_list, pbs=False):
    cur_dir = params_list[2]
    move_dir = params_list[3]
    af_loc = params_list[4]
    Re = params_list[5]
    af_name = params_list[6]
    alpha_min = params_list[7]
    alpha_max = params_list[8]
    alpha_increment = params_list[9]

    with open(move_dir+af_name+'_{}.run'.format(Re), 'w') as fid2:
        fid2.write('load ' + af_name + '.af\n')
        #fid2.write('save {}.af\nY\n'.format(af_name))
        fid2.write('pane\n\n')
        fid2.write('plop\n')
        fid2.write('g\n\noper\niter 1000\n')
        fid2.write('visc {}\n'.format(str(Re)))
        fid2.write('pacc\n\n\n')
        fid2.write('aseq {} {} {}\n'.format(alpha_min, alpha_max, alpha_increment))
        fid2.write('pacc\npwrt\n')
        fid2.write(af_name + '_Re_' + str(Re) + '.polar\n\n\nquit\n')
    
    if pbs:
        with open(move_dir+af_name+'_{}_pbs.cmd'.format(Re), 'w') as fid2:
            pbs_name = 'run_' + af_name
            pbs_cur_dir = 'kourosh/xf/runs/{}/'.format(af_name)
            pbs_cmd = 'xfoil < {}'.format(af_name+'_{}.run'.format(Re))
            fid2.write(pbs.format(pbs_name, pbs_cur_dir, pbs_cmd))
    
    return

def create_data_structure(Re=[400000, 1000000, 50000],
                          alpha_range=[-10, 35, 1.], af_loc=''):

    data_structure = []
    cur_dir = os.getcwd() + '/'
    Re = range(Re[0], Re[1], Re[2])
    if af_loc == '':
        af_loc = cur_dir+'af/'
    files = glob(af_loc + '*.af')
    names = [i.replace(af_loc, '')[:-3] for i in files]
    if not os.path.isdir(cur_dir + 'runs/'):
        os.mkdir(cur_dir + '/runs/')
    for i in names:
        if not os.path.isdir(cur_dir + 'runs/{}/'.format(i)):
            os.mkdir(cur_dir + '/runs/{}/'.format(i))
        xfoil_address = cur_dir + '{}/'.format(i)
        for k in Re:
            data_structure.append([i.replace('af','').replace('.','').replace('_','').replace('/',''), 
                                   cur_dir, 
                                   cur_dir + 'runs/{}/'.format(i),
                                   af_loc, k, i.replace(af_loc, ''),
                                   alpha_range[0], alpha_range[1], alpha_range[2], 0])

    return data_structure

def move_files(af_loc='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/afs/', 
               move_dir='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/',
               airfoils=True, plts=True):
    if airfoils:
        afs = glob(af_loc+'airfoil/*.af')
        for i in afs:
            af_name = i.replace(af_loc+'airfoil/', '')[:-3]
            shutil.copy(i, move_dir + '/runs/{}/'.format(af_name))
    if plts:
        plots = glob(af_loc+'plot/*.png')
        for i in plots:
            af_name = i.replace(af_loc + 'plot/', '')[:-4]
            shutil.copy(i, move_dir + '/runs/{}/'.format(af_name))
    return 

In [4]:
# create the files
# copy the airfoils to the code directory called af the run below
core_num = num_core
data = create_data_structure(Re=[re_min, re_max, 1000000])
df = pd.DataFrame(data)
df.columns=['i','cur_dir', 'run_dir', 'af_loc', 'Re', 'af', 'alpha_min', 'alpha_max', 'alpha_step', 'status']
df['i'] = df['i'].astype(int)
df.sort_values('i', inplace=True)
df = df.reset_index()
df.to_csv('runs.txt', index=False)
for i in df.index:
    create_run_files(df.iloc[i].values)
move_files(af_loc='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/afs/', 
           move_dir='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/')

In [19]:
p = mp.Pool(processes=core_num)
p.imap(xfoil, data)
p.close()
p.join()

'\n#!/bin/bash\n########################################################\n##                                                     #\n## A Simple PBS command file                           #\n## Edit&submit this file using   "qsub pbs_serial.cmd" #\n## Check your job with           "qstat"               #\n## Kill your job with            "qrm" or "qdel"       #\n##                                                     #\n########################################################\n#\n# Specify time limit of your job. (HH:MM:SS)\nPBS -l walltime=00:20:00\nPBS -l nodes=1:procs=1\n\n# Give your job a name so it can be found readily with qstat.\nPBS -N {}\n\n# mail alert at the (e)nd of execution\n#PBS -m e\n#PBS -M your_email_address@ae.metu.edu.tr\n\n# Standard error output merge to the standard output file.\nPBS -j oe\n# Specify the file name for the standard output/error\nPBS -o {}\n\n# Enter your work directory under /cfd\ncd /cfd/{}\n\n# execute your serial program\n./{}\n'